In [1]:
import os
import sys
import pandas as pd
import numpy as np
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chat_models import init_chat_model
from langchain.chains import RetrievalQA


In [2]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = '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'
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "travel_planner_v0"

In [3]:
landmark_prices = pd.read_csv('egypt.csv')
luxor_landmark = pd.read_csv('places.csv')
luxor_restaurants = pd.read_csv('restaurants.csv')
luxor_art = pd.read_csv('artgallery.csv')



In [4]:
all_documents = []
for _, row in landmark_prices.iterrows():
    text = f"""
    Governorate: {row['Governorate/City']}
    Site: {row['Place']}
    Egyptian Ticket: {row['Egyptian']}
    Egyptian Student Ticket: {row['EgyptianStudent']}
    Foreign Ticket: {row['Foreign']}
    Foreign Student Ticket: {row['ForeignStudent']}
    Visiting Times: {row['VisitingTimes']}
    """
    all_documents.append(Document(page_content=text, metadata={"source": 'landmark_prices'}))


In [5]:

for _, row in luxor_landmark.iterrows():
    text = f"""
    Site: {row['title']}
    Description: {row['description']}
    Site Category Name: {row['categoryName']}
    Site Categories: {row['Categories']}
    Location: {row['address']}
    Star Rating: {row['totalScore']}
    """
    all_documents.append(Document(page_content=text, metadata={"source": 'Luxor Landmarks'}))

    





In [6]:
for _, row in luxor_restaurants.iterrows():
    text = f"""
    Name: {row['title']}
    Restaurant Category Name: {row['categoryName']}
    Restaurant Categories: {row['Categories']}
    Address: {row['address']}
    Price Range: {row['price']}
    Star Rating: {row['totalScore']}
    Website: {row['website']}
    """
    all_documents.append(Document(page_content=text, metadata={"source": 'Luxor Restaurants'}))

In [7]:
for _, row in luxor_art.iterrows():
    text = f"""
    Art Gallery Name: {row['title']}
    Art Gallery Categories: {row['Categories']}
    Art Gallery Address: {row['address']}
    Art Gallery Website: {row['website']}
    Star Rating: {row['totalScore']}
    """
    all_documents.append(Document(page_content=text, metadata={"source": 'Luxor Art Galleries'}))

In [8]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")




c:\Users\mosai\anaconda3\envs\ML\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
vectorstore = FAISS.from_documents(all_documents, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 30})


In [10]:
qa_chain = RetrievalQA.from_chain_type(
    llm=init_chat_model("llama-3.3-70b-versatile", model_provider="groq", temperature=0),
    chain_type="stuff",        # "stuff" means it will stuff all retrieved docs into the prompt
    retriever=retriever
)

In [11]:
prompt_template = PromptTemplate(
    input_variables=["context", "user_query"],
    template="""You are a helpful travel planner AI. 
Use the context below, which contains information about 
ticket prices, Places Description and other information.

Context:
{context}

User Query:
{user_query}

Return a 3-day travel itinerary. Include approximate costs and suggestions 
based on the data in the context. 
If you are missing some information, you can make a reasonable assumption, 
but indicate it clearly.
"""
)

In [12]:
def generate_travel_plan(user_query):
    # 1. Retrieve relevant docs
    docs = retriever.get_relevant_documents(user_query)
    # 2. Concatenate doc text
    context_text = "\n".join([doc.page_content for doc in docs])
    # 3. Build the final prompt
    prompt = prompt_template.format(context=context_text, user_query=user_query)
    # 4. Call the LLM
    llm = init_chat_model("llama-3.3-70b-versatile", model_provider="groq", temperature = 0)
    response = llm.invoke(prompt)
    return response

In [14]:
user_query = "Plan a 3-day trip in Luxor with visits to cultural sites and Art and dining(restaurants) options."
travel_plan = generate_travel_plan(user_query)
print(travel_plan)


content="Based on the provided context, I've planned a 3-day trip to Luxor, incorporating cultural site visits, art, and dining options. Please note that some assumptions have been made due to missing information.\n\n**Day 1:**\n\n* Morning: Visit the Luxor Museum (9:00 am - 1:00 pm)\n\t+ Egyptian Ticket: 30 EGP, Foreign Ticket: 400 EGP\n\t+ Audio guide (optional): 30 EGP (Egyptian), 50 EGP (Foreign)\n* Lunch: AHLLAN Restaurant (approximate cost: 200-300 EGP per person, assuming a mid-range price)\n* Afternoon: Explore the Luxor Temple (2:00 pm - 5:00 pm)\n\t+ Egyptian Ticket: 40 EGP, Foreign Ticket: 500 EGP\n* Evening: Enjoy a stroll along the Sphinx Avenue (free, but requires tickets to Karnak and Luxor Temples)\n* Dinner: AHLLAN Restaurant (approximate cost: 200-300 EGP per person)\n\nApproximate cost for Day 1:\nEgyptian: 100 EGP (museum + Luxor Temple), Foreign: 900 EGP (museum + Luxor Temple)\nFood and other expenses: approximately 400-600 EGP per person\n\n**Day 2:**\n\n* Mornin

In [ ]:
travel_plan_qa = qa_chain.invoke(user_query)
print(travel_plan_qa)

C:\Users\mosai\AppData\Local\Temp\ipykernel_2992\3174427764.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  travel_plan_qa = qa_chain.run(user_query)


Here's a suggested 3-day itinerary for a trip to Luxor, incorporating visits to cultural sites, art, and dining options:

Day 1:

* Morning: Visit the Luxor Museum (open from 9:00 am to 1:00 pm and 5:00 pm to 8:00 pm) to get an introduction to the history and culture of Luxor. The Egyptian ticket price is 30 EGP, and the foreign ticket price is 400 EGP.
* Afternoon: Explore the Luxor Temple (open from 6:00 am to 8:00 pm), a magnificent ancient temple complex. The Egyptian ticket price is 40 EGP, and the foreign ticket price is 500 EGP.
* Evening: Dine at AHLLAN Restaurant (rated 4.9 stars), which offers a range of delicious options. You can find the restaurant's location and more information on their website.

Day 2:

* Morning: Visit the Valley of the Kings (open from 6:00 am to 5:00 pm), a vast burial ground for pharaohs and nobles. The Egyptian ticket price is 60 EGP, and the foreign ticket price is 750 EGP.
* Afternoon: Explore the Karnak Temples (open from 6:00 am to 5:00 pm), a v